## Notebook 001


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
             'Yo amo a mi perro',
             'Yo, amo a mi gato',
             'Yo amo a mi rana',
             'Mi gato me ama a mi'
]

tokenizer = Tokenizer(num_words = 100)

In [11]:
tokenizer.fit_on_texts(sentences) #fiteo mi tokenizer
tokenizer.word_index #lo almaceno dentro de un diccionario práctico

{'a': 2,
 'ama': 9,
 'amo': 4,
 'gato': 5,
 'me': 8,
 'mi': 1,
 'perro': 6,
 'rana': 7,
 'yo': 3}

In [0]:
#Donde el word_index es un diccionario en el que se han encodeado las palabras
word_index = tokenizer.word_index

### Padding


In [14]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences #Es en si una lista de vectores donde cada palabra es representada por un número.

[[3, 4, 2, 1, 6], [3, 4, 2, 1, 5], [3, 4, 2, 1, 7], [1, 5, 8, 9, 2, 1]]

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
"""
Como se ve, paddear significa asignar un valor mucho más grande a esos vectores que
de por sí anteriormente he creado. Esto porque siempre tendremos oraciones más largas que otras.
Podemos padear de derecha a izq o viceversa. Todo dependerá de...
"""
padded = pad_sequences(sequences, maxlen=15)
padded

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 2, 1, 6],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 2, 1, 5],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 2, 1, 7],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 8, 9, 2, 1]], dtype=int32)

### Jugando con más features del Tokenizer: 


- El parámetro oov_token nos sirve para codificar palabras que están en el test, pero que no han sido puestas inicialmente en el train. 

Por ejemplo, pondremos de ejemplo train_data y test_data y utilizaremos nuestro parámetro oov junto con el pad que queramos. 


In [0]:
train_data = [
              'Qué bien me siento hoy',
              'Hoy me siento masomenos',
              'Me siento genial!',
              'No me siento tan bien'
]

In [0]:
tokenizer2 = Tokenizer(num_words=100, oov_token="<OOV>")
tokenizer2.fit_on_texts(train_data)

In [27]:
#No olvidar poner el tokenizer a secuencias...
sequences2 = tokenizer2.texts_to_sequences(train_data)
word_index2= tokenizer2.word_index
print(word_index2)
#Ahora tendremos que paddearlo para que todos tengan el mimso length
padded2 = pad_sequences(sequences2, maxlen=10)
print(padded2)
#Darse cuenta que el OOV está encondeado como 1, para ponerlo en el futuro test data

{'<OOV>': 1, 'me': 2, 'siento': 3, 'bien': 4, 'hoy': 5, 'qué': 6, 'masomenos': 7, 'genial': 8, 'no': 9, 'tan': 10}
[[ 0  0  0  0  0  6  4  2  3  5]
 [ 0  0  0  0  0  0  5  2  3  7]
 [ 0  0  0  0  0  0  0  2  3  8]
 [ 0  0  0  0  0  9  2  3 10  4]]


In [0]:
#Crearemos nuestra data de test
test_data = [
             'De verdad me siento muy bien',
             'No me siento tan bien',
             'Me siento genial hoy'
]

In [29]:
test_seq = tokenizer2.texts_to_sequences(test_data)
print('\n Test Sequence = ', test_seq)


 Test Sequence =  [[1, 1, 2, 3, 1, 4], [9, 2, 3, 10, 4], [2, 3, 8, 5]]


In [30]:
test_padded = pad_sequences(test_seq, maxlen=10)
print(test_padded)

[[ 0  0  0  0  1  1  2  3  1  4]
 [ 0  0  0  0  0  9  2  3 10  4]
 [ 0  0  0  0  0  0  2  3  8  5]]


### Jugando con data de sacarcasmo (con una competencia de kaggle) 



In [39]:
!wget --no-check-certificate \
  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
  -O /tmp/sarcasm.json

--2020-02-04 04:14:28--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c13::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.05s   

2020-02-04 04:14:33 (120 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [0]:
import json
with open("/tmp/sarcasm.json","r") as f:
  datastore = json.load(f)

In [0]:
sentences = []
labels = []
urls = []
# Como la data está en json, voy a sacarla a listas para poder procesarla con mi Tokenizer
for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

In [0]:
tokenizer_final = Tokenizer(oov_token="<OOV>")
tokenizer_final.fit_on_texts(sentences)

In [47]:
word_index = tokenizer_final.word_index
print(len(word_index))
print("Como se ve, el word index es muy grande. Existen muchas palabras") 

29657
Como se ve, el word index es muy grande. Existen muchas palabras


In [49]:
sequences_final = tokenizer_final.texts_to_sequences(sentences)
#El padding = post siginifica que paddearás después, es decir, los ceros irán al final
padded = pad_sequences(sequences_final, padding='post')
print(sentences[10])
print(padded[10])
print(padded.shape,'<- Es muy grande!') 

airline passengers tackle man who rushes cockpit in bomb threat
[2925 1680 4721   14   37 4275 6972    5 2095 1103    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
(26709, 40) <- Es muy grande!
